In [2]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import warnings
warnings.filterwarnings("ignore")

In [3]:
#READING CAR CSV FILE
cars = pd.read_csv("CarPrice_Assignment.csv")
cars.head()

FileNotFoundError: File b'CarPrice_Assignment.csv' does not exist

In [ ]:
# Analysis of the data set
print("Dataset_Information:\n", cars.info())
print("Dataset_Size:\n", cars.shape)
print("Dataset_Description:\n",cars.describe())

In [ ]:
#spilting into car_company and car_model
cars["car_company"],cars["car_model"] = cars["CarName"].str.split(" ",1).str
cars.head()

In [ ]:
# Converting to numeric
cars["fueltype"] = cars["fueltype"].map({"gas": 0, "diesel": 1})  # GAS == 0 , DIESEL == 1
cars["aspiration"] = cars["aspiration"].map({"std":0,"turbo":1})   # STD == 0 , TURBO == 1
cars["doornumber"] =cars["doornumber"].map({"two":2,"four":4})     # TWO == 2 , FOUR == 4
cars["drivewheel"] = cars["drivewheel"].map({"rwd":0,"fwd":1})     # rwd == 0 , fwd == 1
cars["enginelocation"] = cars["enginelocation"].map({"front":1,"rear":0})  # front == 1 , rear == 0

In [ ]:
# Drop some columns
cars.drop(columns=["car_ID","CarName","car_model"],inplace=True)
cars.head()

In [ ]:
# creating dummies 
cars = pd.get_dummies(cars)

In [ ]:
cars.shape

In [ ]:
 # correlation using heat map
plt.subplots(figsize=(30,50))
sns.heatmap(cars.corr(), annot = True, cmap="YlGnBu")
plt.show()

In [ ]:
from sklearn.model_selection import train_test_split

# the above line will ensure that the training and test date have same number of rows

df_train, df_test = train_test_split(cars, train_size = 0.7, test_size = 0.3, random_state = 100)

In [ ]:
from sklearn.preprocessing import MinMaxScaler
scaler = MinMaxScaler()

In [ ]:
# Apply scaler() to the numeric columns
num_vars = ["symboling","wheelbase","carlength","carwidth","carheight","curbweight","enginesize","boreratio","stroke","compressionratio","horsepower","peakrpm","citympg","highwaympg","price"]

df_train[num_vars] = scaler.fit_transform(df_train[num_vars])

df_train.head()

In [ ]:
y_train = df_train.pop('price')
X_train = df_train

In [ ]:
# Import RFE and LinearRegression
from sklearn.feature_selection import RFE
from sklearn.linear_model import LinearRegression

In [ ]:
# Running RFE with the output of variable = 15
lm = LinearRegression()
lm.fit(X_train, y_train)

rfe = RFE(lm, 15)           
rfe = rfe.fit(X_train, y_train)

In [ ]:
list(zip(X_train.columns,rfe.support_,rfe.ranking_))

In [ ]:
#values supported
col = X_train.columns[rfe.support_]
col

In [ ]:
# Unsupported values
X_train.columns[~rfe.support_]

In [ ]:
# Generating X_test dataframe with RFE selected variables
X_train_rfe = X_train[col]

In [ ]:
# Addition of a constant variable 
import statsmodels.api as sm  
X_train_rfe = sm.add_constant(X_train_rfe)
lm = sm.OLS(y_train,X_train_rfe).fit()   # Running the linear model
#summary of our linear model
print(lm.summary())

In [ ]:
X_train_new = X_train_rfe.drop(["enginetype_dohcv"], axis = 1)

In [ ]:
# Addition of a constant variable 
import statsmodels.api as sm  
X_train_rfe1 = sm.add_constant(X_train_new)
lm = sm.OLS(y_train,X_train_rfe1).fit()   # Running the linear model
#summary of our linear model
print(lm.summary())

In [ ]:
X_train_new1 = X_train_new.drop(["car_company_porsche"], axis = 1)

In [ ]:
# Addition of a constant variable 
import statsmodels.api as sm  
X_train_rfe2 = sm.add_constant(X_train_new1)
lm = sm.OLS(y_train,X_train_rfe2).fit()   # Running the linear model
#summary of our linear model
print(lm.summary())

In [ ]:
X_train_rfe2.columns

In [ ]:
X_train_vif = X_train_rfe2.drop(['const'], axis=1)

In [ ]:
# Calculating the VIFs for the new model
from statsmodels.stats.outliers_influence import variance_inflation_factor

vif = pd.DataFrame()
X = X_train_vif
vif['Features'] = X.columns
vif['VIF'] = [variance_inflation_factor(X.values, i) for i in range(X.shape[1])]
vif['VIF'] = round(vif['VIF'], 2)
vif = vif.sort_values(by = "VIF", ascending = False)
vif

In [ ]:
X_train_vif1 = X_train_vif.drop(['enginesize'], axis=1)

In [ ]:
# Calculating the VIFs for the new model
from statsmodels.stats.outliers_influence import variance_inflation_factor

vif = pd.DataFrame()
X = X_train_vif1
vif['Features'] = X.columns
vif['VIF'] = [variance_inflation_factor(X.values, i) for i in range(X.shape[1])]
vif['VIF'] = round(vif['VIF'], 2)
vif = vif.sort_values(by = "VIF", ascending = False)
vif

In [ ]:
X_train_vif2 = X_train_vif1.drop(['carwidth'], axis=1)

In [ ]:
# Calculating the VIFs for the new model
from statsmodels.stats.outliers_influence import variance_inflation_factor

vif = pd.DataFrame()
X = X_train_vif2
vif['Features'] = X.columns
vif['VIF'] = [variance_inflation_factor(X.values, i) for i in range(X.shape[1])]
vif['VIF'] = round(vif['VIF'], 2)
vif = vif.sort_values(by = "VIF", ascending = False)
vif

In [ ]:
# Addition of a constant variable 
import statsmodels.api as sm  
X_train_rfe3 = sm.add_constant(X_train_vif2)
lm = sm.OLS(y_train,X_train_rfe3).fit()   # Running the linear model
#Summary of our linear model
print(lm.summary())

In [ ]:
X_train_vif3 = X_train_vif2.drop(['cylindernumber_three'], axis=1)

In [ ]:
# Addition of a constant variable 
import statsmodels.api as sm  
X_train_rfe4 = sm.add_constant(X_train_vif3)
lm = sm.OLS(y_train,X_train_rfe4).fit()   # Running the linear model
#summary of our linear model
print(lm.summary())

In [ ]:
X_train_vif4 = X_train_vif3.drop(['car_company_isuzu'], axis=1)

In [ ]:
# Addition of a constant variable 
import statsmodels.api as sm  
X_train_rfe5 = sm.add_constant(X_train_vif4)
lm = sm.OLS(y_train,X_train_rfe5).fit()   # Running the linear model
#summary of our linear model
print(lm.summary())

In [ ]:
X_train_vif5 = X_train_vif4.drop(['cylindernumber_two'], axis=1)

In [ ]:
# Addition of a constant variable 
import statsmodels.api as sm  
X_train_rfe6 = sm.add_constant(X_train_vif5)
lm = sm.OLS(y_train,X_train_rfe6).fit()   # Running the linear model
#summary of our linear model
print(lm.summary())

In [ ]:
X_train_vif6 = X_train_vif5.drop(['carbody_convertible'], axis=1)

In [ ]:
# Addition of a constant variable 
import statsmodels.api as sm  
X_train_rfe7 = sm.add_constant(X_train_vif6)
lm = sm.OLS(y_train,X_train_rfe7).fit()   # Running the linear model
#summary of our linear model
print(lm.summary())

In [ ]:
X_train_vif7 = X_train_vif6.drop(['cylindernumber_twelve'], axis=1)

In [ ]:
# Addition of a constant variable 
import statsmodels.api as sm  
X_train_rfe8 = sm.add_constant(X_train_vif7)
lm = sm.OLS(y_train,X_train_rfe8).fit()   # Running the linear model
#summary of our linear model
print(lm.summary())

In [ ]:
# Calculating the VIFs for the new model
from statsmodels.stats.outliers_influence import variance_inflation_factor

vif = pd.DataFrame()
X = X_train_vif7
vif['Features'] = X.columns
vif['VIF'] = [variance_inflation_factor(X.values, i) for i in range(X.shape[1])]
vif['VIF'] = round(vif['VIF'], 2)
vif = vif.sort_values(by = "VIF", ascending = False)
vif

In [ ]:
y_train_price = lm.predict(X_train_rfe8)

In [ ]:
# Plotting the histogram for the error terms
fig = plt.figure()
sns.distplot((y_train - y_train_price), bins = 20)
fig.suptitle('Error Terms', fontsize = 20)                  # Histogram Plot heading 
plt.xlabel('Errors', fontsize = 18)                         # X-label of Plot

In [ ]:
num_vars = ["symboling","wheelbase","carlength","carwidth","carheight","curbweight","enginesize","boreratio","stroke","compressionratio","horsepower","peakrpm","citympg","highwaympg","price"]
df_test[num_vars] = scaler.transform(df_test[num_vars])

In [ ]:
y_test = df_test.pop('price')
X_test = df_test

In [ ]:
# Using our model to make predictions.

# Creating new dataframe called X_test_
X_test_new = X_test[X_train_vif7.columns]

# Addition of a constant variable 
X_test_new = sm.add_constant(X_test_new)

In [ ]:
# Using Model to make predictions
y_pred = lm.predict(X_test_new)

In [ ]:
# Plotting y_pred and y_test to understand the spread and to analyse predictions.
fig = plt.figure()
plt.scatter(y_test,y_pred)
fig.suptitle('y_test vs y_pred', fontsize=20)              # Plot heading 
plt.xlabel('y_test', fontsize=18)                          # X-label
plt.ylabel('y_pred', fontsize=16)                          # Y-label